In [ ]:
from google.colab import drive
drive.mount('/content/drive')



import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Concatenate
from tensorflow.keras.applications import EfficientNetB7
import tensorflow as tf

# Check if GPU is available
if tf.test.is_gpu_available():
    print("GPU is available")
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU device not found')
    print('Found GPU at: {}'.format(device_name))
else:
    print("No GPU found")

train_metadata_path = '/content/drive/MyDrive/derm_data/HAM10000_training_metadata.csv'
val_metadata_path = '/content/drive/MyDrive/derm_data/HAM10000_validation_metadata.csv'

train_metadata_df = pd.read_csv(train_metadata_path)
val_metadata_df = pd.read_csv(val_metadata_path)

train_image_dir = '/content/drive/MyDrive/derm_data/HAM10000_training_image_dataset'
val_image_dir = '/content/drive/MyDrive/derm_data/HAM10000_validation_image_dataset'

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Add '.jpg' extension to the image_id column in both train and validation dataframes
train_metadata_df['image_id'] = train_metadata_df['image_id'] + '.jpg'
val_metadata_df['image_id'] = val_metadata_df['image_id'] + '.jpg'

# Create the train generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_metadata_df,
    directory=train_image_dir,
    x_col='image_id',  # Pass the column name as a string
    y_col='dx',  # Pass the column name as a string
    target_size=(225, 300),
    batch_size=32,
    class_mode='categorical'
)

# Create the validation generator
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_metadata_df,
    directory=val_image_dir,
    x_col='image_id',  # Pass the column name as a string
    y_col='dx',  # Pass the column name as a string
    target_size=(225, 300),
    batch_size=32,
    class_mode='categorical'
)

with tf.device('/GPU:0'):
    base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(225, 300, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    model_output = Dense(7, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=model_output)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(
        train_generator,
        epochs=10,
        validation_data=val_generator
    )

loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')